In [54]:
#!pip install setuptools
#!pip install packaging
#!pip install pandas dash
#!pip install pandas dash
#!pip3 install httpx==0.20 dash plotly


In [55]:
import pandas as pd
import dash 
from dash.dependencies import Output,Input
from dash import html,dcc
import plotly.express as px
import datetime as dt
from dash import no_update


In [56]:
app=dash.Dash(__name__)

In [57]:
df =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

In [58]:
df['Month']=pd.to_datetime(df.Date).dt.month_name()
df['Year']=pd.to_datetime(df.Date).dt.year

In [59]:
app.layout=html.Div(
    children=[
    html.H1('Australia Wildfire Dashboard',style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 26}),
    
    html.Div([
        html.Div([
            html.H2('Select Region:',style={'margin-right':'2em'}),
            dcc.RadioItems([{"label":"New South Wales","value": "NSW"},
                                    {"label":"Northern Territory","value": "NT"},
                                    {"label":"Queensland","value": "QL"},
                                    {"label":"South Australia","value": "SA"},
                                    {"label":"Tasmania","value": "TA"},
                                    {"label":"Victoria","value": "VI"},
                                    {"label":"Western Australia","value": "WA"}],'NSW',id='region'),
            ]),
         html.Div([
             html.H2('Select Year',style={'margin-right':'2em'}),
             dcc.Dropdown(df.Year.unique(),value=2005,id='year'),
             ]),
         html.Div([
             html.Div([],id='plot1'),
             html.Div([],id='plot2'),
             ],style={'display':'flex'}),
        
    ]),
    ]
)

In [60]:
@app.callback([Output(component_id='plot1',component_property='children'),Output(component_id='plot2',component_property='children')],
              [Input(component_id='region',component_property='value'),Input(component_id='year',component_property='value')])
def region_year_display(reg,yr):
    region_data = df[df['Region'] == reg]
    data = region_data[region_data['Year']==yr]

    
    est_data = data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
    fig1 = px.pie(est_data, values='Estimated_fire_area', names='Month', title="{} : Monthly Average Estimated Fire Area in year {}".format(reg,yr))   
     #Plot two - Monthly Average Count of Pixels for Presumed Vegetation Fires
    veg_data = data.groupby('Month')['Count'].mean().reset_index()
    fig2 = px.bar(veg_data, x='Month', y='Count', title='{} : Average Count of Pixels for Presumed Vegetation Fires in year {}'.format(reg,yr))  
    
    return[dcc.Graph(figure=fig1),dcc.Graph(figure=fig2)]

In [61]:
app.run_server()